# Run `MTMCSKAT` over *in vitro* transformation GWAS data using residual traits regressed over phase data

This batch is for versions of traits that have been processed by regressing over phase covariates (with `lm`) and collecting residuals. The only covariates used in the SKAT model will be for PCs... no need to include phase covariate data again after regresing over it and collecting residuals.

We are testing this approach alongside the three other approaches to controlling for phase (in notebook b6pt1)

We are trying to employ the approach described here https://vsoch.github.io/lessons/sherlock-jobs/

In [1]:
library(stringr)

## Make a short list of phenotypes to test

In [2]:
pheno_list <- list.files("SKAT_pheno_files_first_05to08/",
                         pattern = "resid\\.header",
                         #pattern = "w\\.header",
                         full.names = TRUE)

In [3]:
pheno_list

[1] "SKAT_pheno_files_first_05to08//Total_signif_pixels_Shoot_CTR_wk10_resid.header.pheno"
[2] "SKAT_pheno_files_first_05to08//callus_transformation_PC1-uv_resid.header.pheno"      
[3] "SKAT_pheno_files_first_05to08//portion_transgenic_Shoot_AS_wk10_resid.header.pheno"  
[4] "SKAT_pheno_files_first_05to08//portion_transgenic_Shoot_AS_wk7_resid.header.pheno"   
[5] "SKAT_pheno_files_first_05to08//shoot_transformation_PC2-uv_resid.header.pheno"

In [4]:
length(pheno_list)

[1] 5

## Make a list of scaffolds to test

In [5]:
scaffold_list <- list.files("split_Chr_2to8parts/",
                            full.names = TRUE)

## Make a short list of covariate files to test

I think SKAT just automatically ignores redundancies so singular matrix is ok

In [6]:
covariate_list <- list.files("covariates_in_vitro",
                             full.names = TRUE)

In [7]:
covariate_list

[1] "covariates_in_vitro/PConly.skatcov"                         
[2] "covariates_in_vitro/PseudoSeasonPhaseIndicatorAndPC.skatcov"
[3] "covariates_in_vitro/SeasonPhaseIndicatorAndPC.skatcov"

In [8]:
library(data.table)

In [1]:
# fread(covariate_list[1])

For traits processed by regressing over phase covariate and collecting residuals, only use PC covariates.

In [10]:
covariate_list <- covariate_list[1]

## Define this function `baseprefix`

In [11]:
baseprefix <- function(filename){
    filename <- basename(filename)
    filename <- unlist(str_split(filename, "\\."))[1]
    return(filename)
}

## Declare other key variables

In [12]:
if(!dir.exists("/expanse/lustre/projects/osu123/naglemi/SKAT_testing/Results_transfGWAS/")){
    dir.create("/expanse/lustre/projects/osu123/naglemi/SKAT_testing/Results_transfGWAS/")
}

In [13]:
window_size <- 3000
window_shift <- 1000
output_dir <- "/expanse/lustre/projects/osu123/naglemi/SKAT_testing/Results_transfGWAS/"
pre_allocated_dir <- "/expanse/lustre/projects/osu123/naglemi/SKAT_testing/new_pre_allocated"

desired_sig_figs <- 2
min_accuracy <- 4
max_accuracy <- 5
plot <- 1
n_thread <- 24
top_N <- 2
missing_cutoff <- 0.15

project <- "osu123"

myenv <- "farmcpupp"

source_path <- "/expanse/lustre/projects/osu123/naglemi/SKAT_testing/mtmcskat"

## Prepare (and submit) job files in a loop

In [14]:
pheno_list

[1] "SKAT_pheno_files_first_05to08//Total_signif_pixels_Shoot_CTR_wk10_resid.header.pheno"
[2] "SKAT_pheno_files_first_05to08//callus_transformation_PC1-uv_resid.header.pheno"      
[3] "SKAT_pheno_files_first_05to08//portion_transgenic_Shoot_AS_wk10_resid.header.pheno"  
[4] "SKAT_pheno_files_first_05to08//portion_transgenic_Shoot_AS_wk7_resid.header.pheno"   
[5] "SKAT_pheno_files_first_05to08//shoot_transformation_PC2-uv_resid.header.pheno"

In [15]:
covariate_list

[1] "covariates_in_vitro/PConly.skatcov"

In [16]:
length(scaffold_list)

[1] 52

In [18]:
for(phenotype in pheno_list){
    for(covariate in covariate_list){
            for(scaffold in scaffold_list){
                
                job_id <- paste0("job-", baseprefix(phenotype),
                                 "-", baseprefix(scaffold),
                                 "-", baseprefix(covariate))
                
                if(!dir.exists(".job")) dir.create(".job")
                
                specific_output_dir <- paste0(output_dir,
                                              "/",
                                              job_id)
                
                output_basename <- paste0(specific_output_dir, "/pheno-",
                            basename(phenotype),
                            "-scaff-",
                            basename(scaffold),
                                        ".csv")
                
                if(file.exists(output_basename)){
                    message(paste("There already exists output for", output_basename), "\nWe will skip this")
                    next
                }
                
                if(!file.exists(output_basename)){
                    message(paste("Will spawn job", output_basename))
                    print(Sys.time())
                }
                
                print("Not skipping...")
                sink(paste0(".job/", job_id, ".job"))
                
                call <- paste0("Rscript ", source_path, "/wrappers/mtmcskat_wrapper.R",
                              " --phenodata=", phenotype,
                              " --covariates=", covariate,
                              " --raw_file_path=", scaffold,
                              " --window_size=", window_size,
                              " --window_shift=", window_shift,
                              " --output_dir=", output_dir,
                              " --pre_allocated_dir=", pre_allocated_dir,
                              " --job_id=", job_id,
                              " --desired_sig_figs=", desired_sig_figs,
                              " --min_accuracy=", min_accuracy,
                              " --max_accuracy=", max_accuracy,
                              " --plot=", 0,
                              " --RAM=", 64000000000,
                              " --n_thread=", n_thread,
                              " --top_N=", top_N,
                              " --missing_cutoff=", missing_cutoff)
                
                
                
                cat("#!/bin/bash\n")
                cat("#SBATCH --cpus-per-task=24\n")
                cat("#SBATCH --ntasks-per-node=1\n")
                cat("#SBATCH --nodes=1\n")
                cat("#SBATCH --partition=compute\n")
                cat(paste0("#SBATCH --job-name=", job_id, "\n"))
                cat(paste0("#SBATCH ", "--output=.job/slurm_", job_id, ".out\n"))
                cat("#SBATCH --time=2:00:00\n")
                cat(paste0("#SBATCH -A ", project, "\n"))
                cat("export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK\n")
                #cat(paste0("conda activate ", myenv, "\n"))
                cat(call)
                
                sink()
                
                system(paste("sbatch",
                             paste0(".job/",job_id,".job")))
                
                #print("Start sleeping at...")
                print(Sys.time())
                Sys.sleep(300)
                cat("\n\n")
        }
    }
}

There already exists output for /expanse/lustre/projects/osu123/naglemi/SKAT_testing/Results_transfGWAS//job-Total_signif_pixels_Shoot_CTR_wk10_resid-1323_cohort_mincount1_defaultmissingrates_Chr10Pt1of2-PConly/pheno-Total_signif_pixels_Shoot_CTR_wk10_resid.header.pheno-scaff-1323_cohort_mincount1_defaultmissingrates_Chr10Pt1of2.snp.pass.traw.gz.csv
We will skip this

There already exists output for /expanse/lustre/projects/osu123/naglemi/SKAT_testing/Results_transfGWAS//job-Total_signif_pixels_Shoot_CTR_wk10_resid-1323_cohort_mincount1_defaultmissingrates_Chr10Pt2of2-PConly/pheno-Total_signif_pixels_Shoot_CTR_wk10_resid.header.pheno-scaff-1323_cohort_mincount1_defaultmissingrates_Chr10Pt2of2.snp.pass.traw.gz.csv
We will skip this

There already exists output for /expanse/lustre/projects/osu123/naglemi/SKAT_testing/Results_transfGWAS//job-Total_signif_pixels_Shoot_CTR_wk10_resid-1323_cohort_mincount1_defaultmissingrates_Chr11Pt1of2-PConly/pheno-Total_signif_pixels_Shoot_CTR_wk10_resid.

[1] "2022-10-15 01:10:21 PDT"
[1] "Not skipping..."
[1] "2022-10-15 01:10:21 PDT"




Will spawn job /expanse/lustre/projects/osu123/naglemi/SKAT_testing/Results_transfGWAS//job-shoot_transformation_PC2-uv_resid-1323_cohort_mincount1_defaultmissingrates_Chr14Pt1of2-PConly/pheno-shoot_transformation_PC2-uv_resid.header.pheno-scaff-1323_cohort_mincount1_defaultmissingrates_Chr14Pt1of2.snp.pass.traw.gz.csv



[1] "2022-10-15 01:15:21 PDT"
[1] "Not skipping..."
[1] "2022-10-15 01:15:21 PDT"




Will spawn job /expanse/lustre/projects/osu123/naglemi/SKAT_testing/Results_transfGWAS//job-shoot_transformation_PC2-uv_resid-1323_cohort_mincount1_defaultmissingrates_Chr14Pt2of2-PConly/pheno-shoot_transformation_PC2-uv_resid.header.pheno-scaff-1323_cohort_mincount1_defaultmissingrates_Chr14Pt2of2.snp.pass.traw.gz.csv



...

Will spawn job /expanse/lustre/projects/osu123/naglemi/SKAT_testing/Results_transfGWAS//job-shoot_transformation_PC2-uv_resid-1323_cohort_mincount1_defaultmissingrates_Chr8Pt2of2-PConly/pheno-shoot_transformation_PC2-uv_resid.header.pheno-scaff-1323_cohort_mincount1_defaultmissingrates_Chr8Pt2of2.snp.pass.traw.gz.csv



[1] "2022-10-15 03:10:26 PDT"
[1] "Not skipping..."
[1] "2022-10-15 03:10:26 PDT"




Will spawn job /expanse/lustre/projects/osu123/naglemi/SKAT_testing/Results_transfGWAS//job-shoot_transformation_PC2-uv_resid-1323_cohort_mincount1_defaultmissingrates_Chr9Pt1of2-PConly/pheno-shoot_transformation_PC2-uv_resid.header.pheno-scaff-1323_cohort_mincount1_defaultmissingrates_Chr9Pt1of2.snp.pass.traw.gz.csv



[1] "2022-10-15 03:15:26 PDT"
[1] "Not skipping..."
[1] "2022-10-15 03:15:27 PDT"




There already exists output for /expanse/lustre/projects/osu123/naglemi/SKAT_testing/Results_transfGWAS//job-shoot_transformation_PC2-uv_resid-1323_cohort_mincount1_defaultmissingrates_Chr9Pt2of2-PConly/pheno-shoot_transformation_PC2-uv_resid.header.pheno-scaff-1323_cohort_mincount1_defaultmissingrates_Chr9Pt2of2.snp.pass.traw.gz.csv
We will skip this



## Notes

Ran out of memory the first time, during resampling

Changed RAM parameter to 64000000000, changed `wiggle_factor` to 10, wiped pre-allocated windows.

Second time, ran out of memory during mtskat.

Output below copied from `slurm_4294967294.out` (backed up as `slurm_4294967294_stashed.out`)

Does this out-of-memory issue only happen on occasions when the scaffold is not pre-allocated? Try re-running the same batch script to find out.

Third attempt, got error easily fixed in commit 00ac4120

Runs out of memory during mtskat if running on whole Chr 7. Try again with split chromosomes, including for Chr 1, the largest. Split into halves only. If still too big, try again with quarters.

New weird error seen 10.05.22: